# Charger les données d'OpenFoodFacts

Nous allons charger les données d'OpenFoodFacts au format CSV avec pandas.

Ce fichier est un export tabulaire de la base de données MongoDB.

In [1]:
import pandas as pd

## Chargement naïf

Attention, le fichier original pèse 2.2 Go (au 2020-01-16).

In [2]:
# modifier le chemin vers le fichier CSV si nécessaire
CSV_FILE = '../data/en.openfoodfacts.org.products.csv'

Le fichier original est téléchargeable sur [le site d'OpenFoodFacts](https://fr.openfoodfacts.org/data), avec les informations techniques minimales permettant de l'ouvrir (encodage et séparateur de champs):

> Le fichier utilise l'encodage Unicode UTF-8.
>
> Le caractère de séparation des champs est `<tab>` (tabulation).

In [3]:
df_off = pd.read_csv(CSV_FILE, sep='\t')

/home/mathieu/miniconda3/envs/off-explore/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,24,25,26,28,44,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Vérifions que le fichier a été correctement chargé, en affichant les premières entrées (5 par défaut).

In [4]:
df_off.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000000000003327986,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1574175736,2019-11-19T15:02:16Z,1574175737,2019-11-19T15:02:17Z,Filetes de pollo empanado,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000100,http://world-en.openfoodfacts.org/product/0000...,del51,1444572561,2015-10-11T14:09:21Z,1444659212,2015-10-12T14:13:32Z,moutarde au moût de raisin,NaN,100g,...,NaN,18.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00000000001111111111,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1560020173,2019-06-08T18:56:13Z,1560020173,2019-06-08T18:56:13Z,Sfiudwx,NaN,dgesc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Combien de colonnes le jeu de données contient-il?

Et combien d'entrées?

In [5]:
df_off.shape

(1110884, 178)

## Occupation mémoire

Le fichier CSV occupe 2.2 Go sur le disque, mais quelle quantité de RAM occupe-t-il en étant chargé dans pandas?

In [6]:
df_off.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110884 entries, 0 to 1110883
Columns: 178 entries, code to carnitine_100g
dtypes: float64(120), int64(2), object(56)
memory usage: 5.4 GB


## Comment diminuer l'occupation mémoire?

### 1. Requalifier des colonnes en variables catégorielles

En première approche, et de façon un peu abusive, on peut assimiler toute colonne contenant moins de 10000 valeurs distinctes (à comparer à 1.1 million d'entrées) à une variable catégorielle.

In [7]:
# on affiche les descriptions des colonnes de type 'object' contenant moins de 10000 valeurs distinctes
for col_name in df_off.columns.values:
    col = df_off[col_name]
    if col.dtype == 'object' and col.nunique() < 10000:
        print(df_off[col_name].describe())
        print()

count      56458
unique      9732
top       France
freq       13690
Name: origins, dtype: object

count      56360
unique      8576
top       france
freq       13997
Name: origins_tags, dtype: object

count                   44858
unique                   2852
top       47.833333,-0.333333
freq                      865
Name: first_packaging_code_geo, dtype: object

count                              48190
unique                              4841
top       sable-sur-sarthe-sarthe-france
freq                                 655
Name: cities_tags, dtype: object

count     126746
unique      9373
top       France
freq       37320
Name: purchase_places, dtype: object

count     1108973
unique       6201
top        France
freq       418931
Name: countries, dtype: object

count       1108971
unique         2258
top       en:france
freq         580796
Name: countries_tags, dtype: object

count     1108971
unique       2258
top        France
freq       580796
Name: countries_en, dtype: object



In [8]:
# on modifie cette boucle pour générer un dictionnaire qui associe à des intitulés de colonnes
# le dtype 'category'
# ex: dtype = {'origins': 'category', 'origins_tags': 'category', ...}
dtype = {}
for col_name in df_off.columns.values:
    col = df_off[col_name]
    if col.dtype == 'object' and col.nunique() < 10000:
        dtype[col_name] = 'category'
dtype

{'origins': 'category',
 'origins_tags': 'category',
 'first_packaging_code_geo': 'category',
 'cities_tags': 'category',
 'purchase_places': 'category',
 'countries': 'category',
 'countries_tags': 'category',
 'countries_en': 'category',
 'traces_tags': 'category',
 'traces_en': 'category',
 'additives': 'category',
 'ingredients_from_palm_oil_tags': 'category',
 'ingredients_that_may_be_from_palm_oil_tags': 'category',
 'nutriscore_grade': 'category',
 'pnns_groups_1': 'category',
 'pnns_groups_2': 'category',
 'states': 'category',
 'states_tags': 'category',
 'states_en': 'category'}

Nous pouvons maintenant recharger le fichier CSV en considérant ces colonnes comme des variables catégorielles.

In [9]:
df_off = pd.read_csv(CSV_FILE, sep='\t', dtype=dtype)
df_off.info(memory_usage='deep')

/home/mathieu/miniconda3/envs/off-explore/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110884 entries, 0 to 1110883
Columns: 178 entries, code to carnitine_100g
dtypes: category(19), float64(120), int64(2), object(37)
memory usage: 3.5 GB


### 2. Diminuer la précision des nombres flottants

In [10]:
# on ajoute ces spécifications au dictionnaire dtype existant
for col_name in df_off.columns.values:
    col = df_off[col_name]
    if col.dtype == 'float64':
        dtype[col_name] = 'float16'
dtype

{'origins': 'category',
 'origins_tags': 'category',
 'first_packaging_code_geo': 'category',
 'cities_tags': 'category',
 'purchase_places': 'category',
 'countries': 'category',
 'countries_tags': 'category',
 'countries_en': 'category',
 'traces_tags': 'category',
 'traces_en': 'category',
 'additives': 'category',
 'ingredients_from_palm_oil_tags': 'category',
 'ingredients_that_may_be_from_palm_oil_tags': 'category',
 'nutriscore_grade': 'category',
 'pnns_groups_1': 'category',
 'pnns_groups_2': 'category',
 'states': 'category',
 'states_tags': 'category',
 'states_en': 'category',
 'cities': 'float16',
 'allergens_en': 'float16',
 'serving_quantity': 'float16',
 'no_nutriments': 'float16',
 'additives_n': 'float16',
 'ingredients_from_palm_oil_n': 'float16',
 'ingredients_from_palm_oil': 'float16',
 'ingredients_that_may_be_from_palm_oil_n': 'float16',
 'ingredients_that_may_be_from_palm_oil': 'float16',
 'nutriscore_score': 'float16',
 'nova_group': 'float16',
 'energy-kj_100g

In [11]:
df_off = pd.read_csv(CSV_FILE, sep='\t', dtype=dtype)
df_off.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1110884 entries, 0 to 1110883
Columns: 178 entries, code to carnitine_100g
dtypes: category(19), float16(120), int64(2), object(37)
memory usage: 2.7 GB


In [12]:
# TODO écrire le dtype final dans un fichier texte dans data/ , qu'on peut charger au début du notebook 2

Nous avons réussi à diviser l'occupation mémoire par deux, de 5.4 Go à 2.7 Go, proche des 2.2 Go du jeu de données sur le disque dur.

Nous verrons dans le notebook suivant comment filtrer un jeu de données pour travailler sur un sous-ensemble.